# Tick History on Jupyter Notebook Quick Start

This article will demonstrate how to request tick history data on demand on Jupyter Notebook.

## What is TRTH?

[TRTH or Thomson Reuters Tick History](https://en.wikipedia.org/wiki/Representational_state_transfer)  is an Internet-hosted product on [DataScope Select platform or DSS](https://developers.refinitiv.com/datascope-select-dss). TRTH is a historical market data service, offering global data dating back to January 1996 . For example, intraday summaries, end of days prices, time and sales, market depth and raw data. TRTH provides  [REST API](https://en.wikipedia.org/wiki/Representational_state_transfer) to access all data. In this article, I will demonstrate how to retreive intraday summaries using an on demand request.

 <img src="https://raw.githubusercontent.com/Refinitiv-API-Samples/Article.TRTH.Python.TRTHOnJupyterNotebookQuickStart/master/figures/TRTH_80.png">


## Prerequisites
- Python 3.6 or higher
- Jupyter Notebook
- DSS username and password which is permissioned for TRTH content. To obtain DSS account, please contact Refinitiv account team for process and details.

## Implementation

The steps and Python source code to request TRTH content on demand are listed below:

### Step 1. Request authentication token with DSS username and password.
 <img src="https://raw.githubusercontent.com/Refinitiv-API-Samples/Article.TRTH.Python.TRTHOnJupyterNotebookQuickStart/master/figures/Step1.png">

- Send HTTP post with DSS username and password

In [1]:
import getpass as gp
import requests
import json

username=input('Enter DSS username:')
password=gp.getpass('Enter DSS Password:')

requestUrl = "https://hosted.datascopeapi.reuters.com/RestApi/v1/Authentication/RequestToken"
requestHeaders={
    "Prefer":"respond-async",
    "Content-Type":"application/json"
    }
requestBody={
    "Credentials": {
    "Username": username,
    "Password": password
  }
}
authenticationResp = requests.post(requestUrl, json=requestBody,headers=requestHeaders)
print("the application received the response for authentication request.")

Enter DSS username:9010004
Enter DSS Password:········
the application received the response for authentication request.


- Check if the status code of the response is 200. If yes, the request has succeeded so the application extracts and prints the authentication token. Otherwise, it prints the error.

In [2]:
if authenticationResp.status_code == 200 :
    jsonResponse = json.loads(authenticationResp.text.encode('ascii', 'ignore'))
    token = jsonResponse["value"]
    print ('Authentication token (valid 24 hours):')
    print (token)
else:
    print("Status Code:",authenticationResp.status_code,"\n Text:",json.dumps(json.loads(authenticationResp.text),indent=4))

Authentication token (valid 24 hours):
_x4k2g-emJDy0DqFLB7DlvYCX3tzxz2YJ0rqoxx95-taZ8NtwZihs0RDN3iPlVERki6aroqhbE1yaOyuC4joyIVNYZ2hpYB7o2aU8em4cNfrUfs0TfXbZNpQ74Xe7Ym9kO5TxugN1VKqnSHMLCpFUDByZ92zMBTXTunCWSuZPTbrG-nzrOq8625I4r49kxFlinyAFU825OH8jGZLZN2y5Kg1aSA4Q83-3KC7O3a0xXTcHuNBmxS3Kuh-JPaZMcklBJFIWkNsS_4DKqUixc6D7-Pj0gATQYAQBnAGUTSsfeIc


### Step 2. Request for data type using the received authentication token 
 <img src="https://raw.githubusercontent.com/Refinitiv-API-Samples/Article.TRTH.Python.TRTHOnJupyterNotebookQuickStart/master/figures/Step2_new.png">


 - Send HTTP post with on demand extraction request to request for data type. 
 
   The applications requests for intraday summaries. For more details of the others tick history data types(reports) and their fields, please see in [Data Dictionary - Custom Reporting](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/docs?content=40731&type=documentation_item)  
  
   You can see all parameters of each data type in **REST API Reference Tree** at **https://hosted.datascopeapi.reuters.com/RestApi.Help/Home/RestApiProgrammingSdk**. You need to login with DSS username and password to access this page.

In [3]:
requestUrl='https://hosted.datascopeapi.reuters.com/RestApi/v1/Extractions/ExtractRaw'
requestHeaders={
    "Prefer":"respond-async",
    "Content-Type":"application/json",
    "Authorization": "token " + token
}
requestBody={
  "ExtractionRequest": {
    "@odata.type": "#ThomsonReuters.Dss.Api.Extractions.ExtractionRequests.TickHistoryIntradaySummariesExtractionRequest",
    "ContentFieldNames": ["Open","High","Low","Last","Volume"],
    "IdentifierList": {
      "@odata.type": "#ThomsonReuters.Dss.Api.Extractions.ExtractionRequests.InstrumentIdentifierList",  
      "InstrumentIdentifiers": [
        {"Identifier": "ADVANC.BK", "IdentifierType": "Ric"},
        {"Identifier": "PTT.BK", "IdentifierType": "Ric"} 
      ],
      "UseUserPreferencesForValidationOptions":"false"    
    },  
    "Condition": {
      "MessageTimeStampIn": "GmtUtc",
      "ReportDateRangeType": "Range",
      "QueryStartDate": "2019-09-03T09:30:00.000Z",
      "QueryEndDate":   "2019-09-03T17:00:00.000Z",
      "SummaryInterval": "FifteenMinutes",
      "TimebarPersistence":"true",
      "DisplaySourceRIC":"true"
    }
  }
}
extractionResp = requests.post(requestUrl, json=requestBody,headers=requestHeaders)
print("the application received the response for on demand extraction request.")

the application received the response for on demand extraction request.


### Step 3. Check the request status  
 <img src="https://raw.githubusercontent.com/Refinitiv-API-Samples/Article.TRTH.Python.TRTHOnJupyterNotebookQuickStart/master/figures/Step3.png">
 
- If the HTTP status code of response is 202 this means the extraction request was accepted, but processing has not completed yet. Hence, the application gets the received location url from 202 response header received in the previous step.

In [4]:
requestStatus =  extractionResp.status_code
print ("HTTP status of the response: " + str(requestStatus))
requestUrl=None
if requestStatus == 202 :
    requestUrl = extractionResp.headers["location"]
    print ('Extraction is not complete, the application will poll the location URL:')
    print (str(requestUrl))

HTTP status of the response: 202
Extraction is not complete, the application will poll the location URL:
https://hosted.datascopeapi.reuters.com/RestApi/v1/Extractions/ExtractRawResult(ExtractionId='0x06cba0180740f97d')


- While the status of the extaction request is 202, poll the request status every 30 seconds using the location url got from the previous step.

In [5]:
import time
while (requestStatus == 202):
    print ('As we received a 202, we wait 30 seconds, then poll again until we do not receive 202')
    time.sleep(30)
    extractionResp = requests.get(requestUrl,headers=requestHeaders)
    requestStatus= extractionResp.status_code
    print ('HTTP status of the response: ' + str(requestStatus))

As we received a 202, we wait 30 seconds, then poll again until we do not receive 202
HTTP status of the response: 200


- When the request is completed(The HTTP stauts code is not 202), check the status code. If it is 200 or OK, the application gets and prints the results which are jobId and the extraction notes. The jobId is used to retrieve the data while the extreaction can be used to analyse data or troubleshooting problems. Apart from the HTTP status code 200, it is an error and the application prints it.

In [6]:
if requestStatus == 200 :
    extractionRespJson = json.loads(extractionResp.text.encode('ascii', 'ignore'))
    jobId = extractionRespJson["JobId"]
    print ('\njobId: ' + jobId + '\n')
    notes = extractionRespJson["Notes"]
    print ('Extraction notes:\n' + notes[0])
else:
    print("Status Code:",extractionResp.status_code,"\n Text:",json.dumps(json.loads(extractionResp.text),indent=4))


jobId: 0x06cba0180740f97d

Extraction notes:
Extraction Services Version 13.1.40889 (ec84d57d2aa3), Built Jul 17 2019 13:34:00
User ID: 9010004
Extraction ID: 2000000100128338
Schedule: 0x06cba0180740f97d (ID = 0x0000000000000000)
Input List (1 items):  (ID = 0x06cba0180740f97d) Created: 09/19/2019 09:28:42 Last Modified: 09/19/2019 09:28:42
Report Template (5 fields): _OnD_0x06cba0180740f97d (ID = 0x06cba0180970f97d) Created: 09/19/2019 09:27:07 Last Modified: 09/19/2019 09:27:07
Schedule dispatched via message queue (0x06cba0180740f97d), Data source identifier (DA265F29E6074240812FA665E7FCFABB)
Schedule Time: 09/19/2019 09:27:09
Processing started at 09/19/2019 09:27:09
Processing completed successfully at 09/19/2019 09:28:42
Extraction finished at 09/19/2019 02:28:42 UTC, with servers: tm04n01, TRTH (63.361 secs)
Instrument <RIC,PTT.BK> expanded to 1 RIC: PTT.BK.
Total instruments after instrument expansion = 1
Quota Message: INFO: Tick History Cash Quota Count Before Extraction: 5

### Step 4. Retrieve data
 <img src="https://raw.githubusercontent.com/Refinitiv-API-Samples/Article.TRTH.Python.TRTHOnJupyterNotebookQuickStart/master/figures/Step4_complete.png">


- Send HTTP get with a JobID got from the 200 OK response to retrieve data from TRTH or AWS 

   TRTH provides downloading some extraction data directly from Amazon Web Services(AWS) where the data files are hosted. The tick history data types which are supported by this features are:
    * Time and Sales
    * Market Depth
    * Intraday Summaries
    * Raw.

  This sample requests for intraday summaries which AWS download supports.  Therefore, I will download data from AWS which delivers faster downloads than TRTH. The application must include the HTTP header field X-Direct-Download true to specify for the download to occur through AWS. When the application sends the requst with X-Direct-Download: true header, it will receive a response with HTTP status 302 or redirect. The response header contains a redirection URI in item Location. Fortunately, Python application automatically follow the redirection so the appliation have nothing to do. A call is made in the background to this URI and the data is retrieved. 
  If you application does not perform redirection automatically, please follows [Advisory: Directly Downloading from Amazon Generates Error](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/docs?content=25898&type=documentation_item) how to perfrom redirection manually and to protect the error from AWS download.   

In [7]:
DownloadFromAWS = False
requestUrl = "https://hosted.datascopeapi.reuters.com/RestApi/v1/Extractions/RawExtractionResults" + "('" + jobId + "')" + "/$value"
requestHeaders={
        "Prefer":"respond-async",
        "Content-Type":"text/plain",
        "Accept-Encoding":"gzip",
        "Authorization": "token " + token
}
if DownloadFromAWS==True:
    requestHeaders.update({"X-Direct-Download":"true"})
dataRetrieveResp=requests.get(requestUrl,headers=requestHeaders,stream=True)
print("the application received the response for retreiving data from the jobId.")

the application received the response for retreiving data from the jobId.


- If the status is 200 or OK that means the application can retreive data from TRTH or AWS sucessfully. Otherwise, it prints the error. 

In [ ]:
if dataRetrieveResp.status_code == 200 :
    print ('The application retrieve data from the server successfully.')
else:
    print("Status Code:",extractionResp.status_code,"\n Text:",json.dumps(json.loads(extractionResp.text),indent=4))

- To avoid data lost issues especially with large data sets. The application should dowload compressed data and save it before decompressing it instead of decompressing it on the fly. For more information, please refer to [Advisory: Avoid Incomplete Output - Download then Decompress](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/docs?content=22738&type=documentation_item)

   In Python, you can ensure that the data is not automatically decompressed on the fly by setting *Response.raw.decode_content* to be *false*.

In [8]:
import os
import shutil
dataRetrieveResp.raw.decode_content = False
fileName= os.getcwd() + "\compressData.csv.gz" 
print ('Saving compressed data to file:' + fileName + ' ... please be patient')

chunk_size = 1024
rr = dataRetrieveResp.raw
with open(fileName, 'wb') as fd:
    shutil.copyfileobj(rr, fd, chunk_size)
fd.close

print ('Finished saving compressed data to file:' + fileName + '\n')

Saving compressed data to file:C:\Users\u8007607\JupyterNoteBook\TRTH_Python_JupyterNotebook\compressData.csv.gz ... please be patient
Finished saving compressed data to file:C:\Users\u8007607\JupyterNoteBook\TRTH_Python_JupyterNotebook\compressData.csv.gz



- In production, you should handle the data line by line instead of store all the data in one variable. This is to avoid issues with large data sets.
  
  To display some data lines, the application reads and decompress some lines in the data file just created.

In [9]:
import gzip
maxLines = 10
print ('Read data from file, and decompress at most ' + str(maxLines) + ' lines of it:')
uncompressedData = ""
count = 0
with gzip.open(fileName, 'rb') as fd:
    for line in fd:
        dataLine = line.decode("utf-8")
        print (dataLine)
        uncompressedData = uncompressedData + dataLine
        count += 1
        if count >= maxLines:
            break
fd.close()

Read data from file, and decompress at most 10 lines of it:
#RIC,Alias Underlying RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume

PTT.BK,,Market Price,2019-09-03T02:30:00.000000000Z,+7,Intraday 15Min,,,,,

PTT.BK,,Market Price,2019-09-03T02:45:00.000000000Z,+7,Intraday 15Min,43.5,43.75,43.5,43.75,463600

PTT.BK,,Market Price,2019-09-03T03:00:00.000000000Z,+7,Intraday 15Min,43.5,43.75,43.25,43.5,4247200

PTT.BK,,Market Price,2019-09-03T03:15:00.000000000Z,+7,Intraday 15Min,43.5,43.5,43.25,43.25,431900

PTT.BK,,Market Price,2019-09-03T03:30:00.000000000Z,+7,Intraday 15Min,43.5,43.5,43.25,43.25,453600

PTT.BK,,Market Price,2019-09-03T03:45:00.000000000Z,+7,Intraday 15Min,43.25,43.5,43.25,43.5,790900

PTT.BK,,Market Price,2019-09-03T04:00:00.000000000Z,+7,Intraday 15Min,43.25,43.5,43.25,43.5,128700

PTT.BK,,Market Price,2019-09-03T04:15:00.000000000Z,+7,Intraday 15Min,43.5,43.5,43.25,43.25,218200

PTT.BK,,Market Price,2019-09-03T04:30:00.000000000Z,+7,Intraday 15Min,43.25,4

- Display the same data as the previous step in a pretty format(table) using a pandas dataframe. This is optional step.

In [10]:
from io import StringIO
import pandas as pd

timeSeries = pd.read_csv(StringIO(uncompressedData))
timeSeries

,#RIC,Alias Underlying RIC,Domain,Date-Time,GMT Offset,Type,Open,High,Low,Last,Volume
0,PTT.BK,NaN,Market Price,2019-09-03T02:30:00.000000000Z,7,Intraday 15Min,NaN,NaN,NaN,NaN,NaN
1,PTT.BK,NaN,Market Price,2019-09-03T02:45:00.000000000Z,7,Intraday 15Min,43.50,43.75,43.50,43.75,463600.0
2,PTT.BK,NaN,Market Price,2019-09-03T03:00:00.000000000Z,7,Intraday 15Min,43.50,43.75,43.25,43.50,4247200.0
3,PTT.BK,NaN,Market Price,2019-09-03T03:15:00.000000000Z,7,Intraday 15Min,43.50,43.50,43.25,43.25,431900.0
4,PTT.BK,NaN,Market Price,2019-09-03T03:30:00.000000000Z,7,Intraday 15Min,43.50,43.50,43.25,43.25,453600.0
5,PTT.BK,NaN,Market Price,2019-09-03T03:45:00.000000000Z,7,Intraday 15Min,43.25,43.50,43.25,43.50,790900.0
6,PTT.BK,NaN,Market Price,2019-09-03T04:00:00.000000000Z,7,Intraday 15Min,43.25,43.50,43.25,43.50,128700.0
7,PTT.BK,NaN,Market Price,2019-09-03T04:15:00.000000000Z,7,Intraday 15Min,43.50,43.50,43.25,43.25,218200.0
8,PTT.BK,NaN,Market Price,2019-09-03T04:30:00.000000000Z,7,Intraday 15Min,43.25,43.50,43.25,43.50,2358900.0


# References
- [TRTH or Thomson Reuters Tick History](https://en.wikipedia.org/wiki/Representational_state_transfer) 
- [DataScope Select platform or DSS](https://developers.refinitiv.com/datascope-select-dss)
- [REST API](https://en.wikipedia.org/wiki/Representational_state_transfer)
- [REST API Reference Tree](https://hosted.datascopeapi.reuters.com/RestApi.Help/Home/RestApiProgrammingSdk)
- [Advisory: Directly Downloading from Amazon Generates Error](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/docs?content=25898&type=documentation_item)
- [Advisory: Avoid Incomplete Output - Download then Decompress](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/docs?content=22738&type=documentation_item)
- [REST API Tutorial 3: On Demand data extraction workflow](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/learning?content=11307&type=learning_material_item)
- [REST API Tutorial 6: On Demand intraday bars extraction](https://developers.refinitiv.com/thomson-reuters-tick-history-trth/thomson-reuters-tick-history-trth-rest-api/learning?content=11243&type=learning_material_item)